# Parkinson’s UPDRS Progression Modeling (Regression)

This notebook adapts your dataset (columns like `visit_month`, `updrs_1..4`) into a **regression ML project**.

## What you have
- Longitudinal clinical visits per patient
- Targets you can predict: `updrs_1`, `updrs_2`, `updrs_3`, `updrs_4` (severity scores)

## What we’ll do
1. Load the CSV from `data/`
2. Basic EDA + sanity checks
3. Create a regression target (default: `updrs_3`)
4. Split data (group-aware split by `patient_id` recommended)
5. Train models (LinearRegression, Ridge, RandomForestRegressor)
6. Evaluate (MAE, RMSE, R²) + plots

**Tip:** If you want to predict a different UPDRS score, change `TARGET = 'updrs_3'`.


In [1]:
!pip -q install numpy pandas scikit-learn matplotlib

In [2]:
import os, glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)

print('Imports ready')

Imports ready


## 1) Load data
Put your CSV inside `data/`.


In [3]:
DATA_DIR = 'data'
csv_files = sorted(glob.glob(os.path.join(DATA_DIR, '*.csv')))
if not csv_files:
    raise FileNotFoundError(f"No CSV found in '{DATA_DIR}/'. Put your dataset CSV there.")

csv_path = csv_files[0]
print('Using:', csv_path)
df = pd.read_csv(csv_path)
df.head()

Using: data/supplemental_clinical_data.csv


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


In [4]:
print('Shape:', df.shape)
df.info()

Shape: (2223, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2223 non-null   object 
 1   patient_id                           2223 non-null   int64  
 2   visit_month                          2223 non-null   int64  
 3   updrs_1                              2010 non-null   float64
 4   updrs_2                              2009 non-null   float64
 5   updrs_3                              2218 non-null   float64
 6   updrs_4                              1295 non-null   float64
 7   upd23b_clinical_state_on_medication  1122 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 139.1+ KB


In [5]:
# Quick missing values overview
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0].head(20) if (missing > 0).any() else 'No missing values found.'

upd23b_clinical_state_on_medication    1101
updrs_4                                 928
updrs_2                                 214
updrs_1                                 213
updrs_3                                   5
dtype: int64

## 2) Choose a target
Your dataset doesn’t have a `status` label (healthy vs PD). Instead, it has **UPDRS severity scores**.

We’ll default to predicting **motor severity**: `updrs_3`.


In [6]:
TARGET = 'updrs_3'  # change to 'updrs_1' / 'updrs_2' / 'updrs_4' if you want

required_cols = {'patient_id', 'visit_month', TARGET}
missing_req = required_cols - set(df.columns)
if missing_req:
    raise ValueError(f"Missing required columns: {missing_req}. Your columns: {list(df.columns)[:25]}...")

df[TARGET].describe()

count    2218.000000
mean       22.917944
std        12.342596
min         0.000000
25%        14.000000
50%        22.000000
75%        31.000000
max        72.000000
Name: updrs_3, dtype: float64

## 3) Feature engineering

Basic approach:
- Use numeric predictors like `visit_month`, other UPDRS scores, and medication state columns
- Drop ID columns from features (`visit_id`, `patient_id`)

### Leakage warning
If you predict `updrs_3` and include `updrs_3` in features, that’s cheating. We will drop the target from X.

Also, **including other UPDRS scores (`updrs_1`, `updrs_2`, `updrs_4`) may be OK** depending on your goal:
- If you want to estimate `updrs_3` from other clinical measures taken at the same visit, it’s fine.
- If you want to forecast future `updrs_3`, you should use only past information.

This notebook does the simpler “same-visit” regression.


In [7]:
# Build X/y
drop_cols = ['visit_id'] if 'visit_id' in df.columns else []

y = df[TARGET].astype(float)
X = df.drop(columns=[TARGET] + drop_cols)

# Keep patient_id for group split, but NOT as a feature
groups = X['patient_id']

# Drop identifiers from features
X = X.drop(columns=['patient_id'])

# ---- Fix non-numeric columns ----
# The medication-state column can contain strings like "On"/"Off" plus NaNs.
# Convert to numeric so the median imputer + models can run.
med_col = 'upd23b_clinical_state_on_medication'
if med_col in X.columns:
    X[med_col] = (
        X[med_col]
        .astype('string')
        .str.strip()
        .map({'On': 1, 'Off': 0})
        .fillna(0)
        .astype(float)
    )

# If any object columns remain, show them (you can decide how to encode them)
obj_cols = X.select_dtypes(include=['object']).columns.tolist()
if obj_cols:
    print('Warning: non-numeric feature columns still present:', obj_cols)

X.head()


,visit_month,updrs_1,updrs_2,updrs_4,upd23b_clinical_state_on_medication
0,0,5.0,3.0,0.0,NaN
1,36,6.0,4.0,0.0,NaN
2,0,4.0,6.0,0.0,NaN
3,36,1.0,8.0,0.0,On
4,0,NaN,NaN,NaN,NaN


In [ ]:
# ---- FIX NON-NUMERIC COLUMNS (AUTO-GENERATED FIX) ----

# Drop visit_id if present
if 'visit_id' in X.columns:
    X = X.drop(columns=['visit_id'])

# Encode medication state: On=1, Off/NaN=0
if 'upd23b_clinical_state_on_medication' in X.columns:
    X['upd23b_clinical_state_on_medication'] = (
        X['upd23b_clinical_state_on_medication']
        .map({'On': 1, 'Off': 0})
        .fillna(0)
    )

X.dtypes


## 4) Train/test split

Because you have repeated visits per patient, a random split can leak patient information.

**Better:** Group-aware split so the same patient doesn’t appear in both train and test.


In [8]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()

X_train.shape, X_test.shape

((1774, 5), (449, 5))

## 5) Models

We’ll try:
- LinearRegression (baseline)
- Ridge (helps with multicollinearity)
- RandomForestRegressor (nonlinear, often strong baseline)

### Preprocessing pipeline
- Impute missing values (median)
- Standardize (for linear models)


In [10]:
# Convert On/Off strings to 1/0 if present
for col in X.columns:
    if X[col].dtype == "object":
        # common mapping for medication state
        X[col] = X[col].str.strip().map({"On": 1, "Off": 0})

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Identify column types
num_cols = X.select_dtypes(include=["number"]).columns
cat_cols = X.select_dtypes(exclude=["number"]).columns

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

linear = Pipeline([("preprocess", preprocess), ("model", LinearRegression())])
ridge  = Pipeline([("preprocess", preprocess), ("model", Ridge(alpha=1.0, random_state=42))])
rf     = Pipeline([("preprocess", preprocess), ("model", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))])

In [14]:
df.select_dtypes(include=["object"]).head()

,visit_id,upd23b_clinical_state_on_medication
0,35_0,NaN
1,35_36,NaN
2,75_0,NaN
3,75_36,On
4,155_0,NaN


In [15]:
# ---- FIX NON-NUMERIC COLUMNS ----

# 1) Drop visit_id (identifier, not useful for ML)
if 'visit_id' in X.columns:
    X = X.drop(columns=['visit_id'])

# 2) Encode medication state: On = 1, Off/NaN = 0
if 'upd23b_clinical_state_on_medication' in X.columns:
    X['upd23b_clinical_state_on_medication'] = (
        X['upd23b_clinical_state_on_medication']
        .map({'On': 1, 'Off': 0})
        .fillna(0)
    )

# Sanity check
X.dtypes

visit_month                              int64
updrs_1                                float64
updrs_2                                float64
updrs_4                                float64
upd23b_clinical_state_on_medication    float64
dtype: object

In [16]:
def eval_regression(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

linear = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

ridge = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', Ridge(alpha=1.0, random_state=42))
])

rf = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('model', RandomForestRegressor(
        n_estimators=400,
        random_state=42,
        n_jobs=-1
    ))
])

models = {
    'LinearRegression': linear,
    'Ridge': ridge,
    'RandomForest': rf
}

results = []
pred_store = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae, rmse, r2 = eval_regression(y_test, preds)
    results.append((name, mae, rmse, r2))
    pred_store[name] = preds

pd.DataFrame(results, columns=['model', 'MAE', 'RMSE', 'R2']).sort_values('MAE')

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'On'

## 6) Plot predicted vs actual
Good models should align near the diagonal.


In [ ]:
best_name = pd.DataFrame(results, columns=['model','MAE','RMSE','R2']).sort_values('MAE').iloc[0]['model']
best_preds = pred_store[best_name]
print('Best by MAE:', best_name)

plt.figure(figsize=(6, 6))
plt.scatter(y_test, best_preds, alpha=0.6)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'Predicted vs Actual ({best_name}) - Target: {TARGET}')

min_v = min(y_test.min(), best_preds.min())
max_v = max(y_test.max(), best_preds.max())
plt.plot([min_v, max_v], [min_v, max_v])
plt.show()

## 7) Feature importance (RandomForest)

This is quick and useful to learn what drives predictions.


In [ ]:
# Extract feature importances from the fitted RF inside the pipeline
rf_fitted = models['RandomForest']
rf_fitted.fit(X_train, y_train)
rf_model = rf_fitted.named_steps['model']

importances = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
importances.head(15)

In [ ]:
plt.figure(figsize=(8, 5))
importances.head(15).sort_values().plot(kind='barh')
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances (RandomForest)')
plt.show()

## 8) Next upgrades (optional)

### Make it a true forecasting problem
Instead of predicting UPDRS at the same visit, create a target like:
- `updrs_3_next = updrs_3` shifted to the next visit per patient
and train using only information from the current/previous visits.

### Better evaluation
- Use GroupKFold cross-validation (patient-level)
- Add Gradient Boosting models (XGBoost/LightGBM if you want)
